# Домашняя работа по теме "Машинное обучение ранжированию"

В этом ДЗ мы:
- научимся работать со стандартным датасетом для машинного обучения ранжированию [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)
- попробуем применить на практике все то, чему мы научились на семинаре

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [ ]:
import pathlib
from timeit import default_timer as timer

import numpy as np
import pandas as pd

from catboost import datasets, utils

## Датасет MSLR (Microsoft Learning to Rank)

Загрузим датасет MSLR.

Полный датасет можно скачать с официального сайта: https://www.microsoft.com/en-us/research/project/mslr/

Строго говоря, он состоит их 2х частей:

- основной датасет MSLR-WEB30K -- он содержит более 30 тыс. запросов
- "маленький" датасет MSLR-WEB10K, который содержит только 10 тыс. запросов и является случайным сэмплом датасета MSLR-WEB30K

в этом ДЗ мы будем работать с MSLR-WEB10K, т.к. полная версия датасета может просто не поместиться у нас в RAM (и, тем более, в память видеокарты если мы учимся на GPU)

Будем считать, что мы самостоятельно скачали датасет с официального сайта, поместили его в папку КОРЕНЬ-ЭТОЙ-РЕПЫ/data/mslr-web10k и раззиповали.

В результате у нас должна получиться следующая структура папок:

In [ ]:
!ls ../../data

In [ ]:
!ls ../../data/mslr-web10k

Заметим, что датасет довольно большой, он весит 7.7 GB.

Он состоит из нескольких фолдов, которые по сути представляют из себя разные разбиения одних и тех же данных на обучающее, валидационное и тестовые множеста.

Дальше мы будем использовать только первый фолд: Fold1.

Заглянем внутрь:

In [ ]:
!ls ../../data/mslr-web10k/Fold1

Видим, что у нас 3 файла с говорящими названиями, соответсвующими сплитам нашего датасета.

Посмотрим на содержимое одного из файлов:

In [ ]:
!head -n 1 ../../data/mslr-web10k/Fold1/train.txt

Видим, что данные лежат в знакомом нам по семинару формате:

- В первой колонке лежит таргет (оценка асессора), по 5-балльной шкале релевантности: от 0 до 4 (включительно)
- Во второй колонке лежит ID запроса, по которому можно сгруппировать все оценки документов в рамках одного и того же запроса
- Дальше идет вектор из 128 фичей (таких как значения BM25 и т.п.), их точная природа нам сейчас на важна

В файле qid и все-фичи кодируются в формате КЛЮЧ:ЗНАЧЕНИЕ, напр. 130:116 -- тут 130 это номер фичи, а 116 -- ее значение.

Такой формат в мире машинного обучения часто называют svm light формат (в честь когда-то популярной библиотеки SVM-Light)

Напишем немного вспомогательного кода для загрузки этого датасета:

In [ ]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns
    
def load_svmlight_file(input_file, max_num_lines=0):
    """Loads dataset split in SVM-Light format"""
    def _parse_field(field):
        parts = field.split(':')
        if len(parts) != 2:
            raise Exception(f"invalid number of parts in field {field}")
        return parts

    num_features = 136
    exp_num_fields = num_features + 2
    num_lines = 0
    X = []
    with open(input_file, 'rt') as f:
        for line in f:
            try:
                num_lines += 1
                                  
                # Parse into fields
                fields = line.rstrip().split(' ')
                num_fields = len(fields)
                if num_fields != exp_num_fields:
                    raise Exception(f"invalid number of fields {num_fields}")
    
                # Parse every field
                label = int(fields[0])
                _, qid_str = _parse_field(fields[1])
                qid = int(qid_str)
                x = [label, qid]
                for field in fields[2:]:
                    _, feature_str = _parse_field(field)
                    feature = float(feature_str)
                    x.append(feature)
    
                # Add new object
                X.append(x)
                if num_lines % 50000 == 0:
                    print(num_lines)
                if max_num_lines > 0 and num_lines == max_num_lines:
                    print(f"WARNING: stop loading, line limit reached: max_num_lines = {max_num_lines} input_file = {input_file}")
                    break
            except Exception as e:
                raise Exception(f"error at line {num_lines} in {input_file}") from e
    
    # To pandas
    X = np.asarray(X)
    df = pd.DataFrame(X, columns=generate_column_names(num_features))
    print(f"Loaded SVM-Light file {input_file}")
    return df

И теперь загрузим датасет:

In [ ]:
fold_dir = pathlib.Path("../../data/mslr-web10k/Fold1")

df_train = load_svmlight_file(fold_dir.joinpath("train.txt"), max_num_lines=20000)
df_valid = load_svmlight_file(fold_dir.joinpath("vali.txt"), max_num_lines=5000)
df_test = load_svmlight_file(fold_dir.joinpath("test.txt"), max_num_lines=5000)

Посмотрим на данные:

In [ ]:
print(df_train.head(5))

Т.е. теперь мы видим что данные доступны в точно таком же виде, как это было в семинаре.

Проведем небольшой EDA.

Всего у нас 20000 документов в трейне:

In [ ]:
print(df_train.info())

5000 документов в валидации:

In [ ]:
print(df_valid.info())

И 5000 документов в тесте:

In [ ]:
print(df_test.info())

Сколько у нас запросов?

In [ ]:
num_queries_train = df_train['qid'].nunique()
num_queries_valid = df_valid['qid'].nunique()
num_queries_test = df_test['qid'].nunique()
print(f"Got {num_queries_train} train, {num_queries_valid} valid and {num_queries_test} test queries")

## Обучаем модель

Теперь можно приступить непосредственно к обучению модели. 

Объявим класс модели, который надо будем заимлементить в этом ДЗ:

In [ ]:
class Model:
    def __init__(self):
        pass

    def fit(self, df_train, df_valid):
        pass

    def predict(self, df_test):
        return np.random.rand(len(df_test))

Создадим и применим модель:

In [ ]:
# Create model
model = Model()

# Fit
start = timer()
model.fit(df_train, df_valid)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f}")

# Predict
y_hat_test = model.predict(df_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

Теперь, имея предикты, можно посчитать метрики качества:

In [ ]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")

# Get test targets and groups
y_test = df_test['label'].to_numpy()
q_test = df_test['qid'].to_numpy().astype('uint32')
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

Ожидаем, что ваша модель покажет результаты выше бейзлайна!